# Create SRA submission sheet
This Python Jupyter notebook creates a BioSample submission sheet for the Sequence Read Archive:

First, import Python modules:

In [1]:
import datetime
import glob
import itertools
import os

import natsort

import pandas as pd

import yaml

Display full columns of data frames:

In [2]:
pd.set_option('display.max_colwidth', None)

Read the configuration for the uploads:

In [3]:
with open('upload_config.yaml') as f:
    config = yaml.safe_load(f)

Read the Illumina runs to submit:

In [4]:
print(f"Reading Illumina runs from {config['barcode_runs']}")

illumina_runs = pd.read_csv(config['barcode_runs'])

print('Here are the first few entries in the Illumina runs:')
illumina_runs.head()

Reading Illumina runs from ../data/barcode_runs.csv
Here are the first few entries in the Illumina runs:


,library,sample,date,experiment,antibody,concentration,concentration_units,group,selection,frac_escape,cells_sorted,R1
0,lib1,expt_149-151-none-0-reference,220128,expt_149-151,none,0,ng_per_mL,Karolinska,reference,NaN,NaN,/shared/ngs/illumina/agreaney/220203_VH00319_144_AAATNGMM5/Unaligned/Project_agreaney/expt_149-151_lib1_ref*.fastq.gz
1,lib2,expt_149-151-none-0-reference,220128,expt_149-151,none,0,ng_per_mL,Karolinska,reference,NaN,NaN,/shared/ngs/illumina/agreaney/220203_VH00319_144_AAATNGMM5/Unaligned/Project_agreaney/expt_149-151_lib2_ref*.fastq.gz
2,lib1,expt_149-CAB-A17-139-escape,220128,expt_149,CAB-A17,139,ng_per_mL,Karolinska,escape,0.119,1113047.0,/shared/ngs/illumina/agreaney/220218_VH00699_65_AAAW23WM5/Unaligned/Project_agreaney/expt_149_lib1_abneg*.fastq.gz
3,lib2,expt_149-CAB-A17-139-escape,220128,expt_149,CAB-A17,139,ng_per_mL,Karolinska,escape,0.116,1119273.0,/shared/ngs/illumina/agreaney/220218_VH00699_65_AAAW23WM5/Unaligned/Project_agreaney/expt_149_lib2_abneg*.fastq.gz
4,lib1,expt_150-CAB-A49-222-escape,220128,expt_150,CAB-A49,222,ng_per_mL,Karolinska,escape,0.136,1415076.0,/shared/ngs/illumina/agreaney/220218_VH00699_65_AAAW23WM5/Unaligned/Project_agreaney/expt_150_lib1_abneg*.fastq.gz


Next make submission entries for the barcode runs, initially in "tidy" format with one FASTQ file per row:

In [5]:
submissions_tidy = (
    illumina_runs
    .assign(
        biosample_accession=lambda x: config['biosample_accession'],
        library_ID=lambda x: x['library'] + '_' + x['sample'],
        title=lambda x: 'SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for ' + x['sample'],
        library_strategy='AMPLICON',
        library_source='SYNTHETIC',
        library_selection='PCR',
        library_layout='single',
        platform='ILLUMINA',
        instrument_model='Illumina HiSeq 2500',
        design_description='PCR of barcodes from RBD variants',
        filetype='fastq',
        filename_fullpath=lambda x: x['R1'].str.split(';')
                                    .map(lambda flist: list(itertools.chain.from_iterable(glob.glob(f) for f in flist))),       
        )
    .explode('filename_fullpath')
    .assign(filename_fullpath=lambda x: x['filename_fullpath'].str.strip(),
            filename=lambda x: x['filename_fullpath'].map(os.path.basename))
    .drop(columns=illumina_runs.columns)
    .reset_index(drop=True)
    )

assert submissions_tidy['filename_fullpath'].map(os.path.isfile).all()

print('Here are the first few submission entries in tidy format:')
submissions_tidy.head()

Here are the first few submission entries in tidy format:


,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename_fullpath,filename
0,SAMN26873633,lib1_expt_149-151-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_149-151-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/220203_VH00319_144_AAATNGMM5/Unaligned/Project_agreaney/expt_149-151_lib1_ref-3_S3_R1_001.fastq.gz,expt_149-151_lib1_ref-3_S3_R1_001.fastq.gz
1,SAMN26873633,lib1_expt_149-151-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_149-151-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/220203_VH00319_144_AAATNGMM5/Unaligned/Project_agreaney/expt_149-151_lib1_ref-1_S1_R1_001.fastq.gz,expt_149-151_lib1_ref-1_S1_R1_001.fastq.gz
2,SAMN26873633,lib1_expt_149-151-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_149-151-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/220203_VH00319_144_AAATNGMM5/Unaligned/Project_agreaney/expt_149-151_lib1_ref-2_S2_R1_001.fastq.gz,expt_149-151_lib1_ref-2_S2_R1_001.fastq.gz
3,SAMN26873633,lib1_expt_149-151-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_149-151-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/220203_VH00319_144_AAATNGMM5/Unaligned/Project_agreaney/expt_149-151_lib1_ref-4_S4_R1_001.fastq.gz,expt_149-151_lib1_ref-4_S4_R1_001.fastq.gz
4,SAMN26873633,lib2_expt_149-151-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_149-151-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/220203_VH00319_144_AAATNGMM5/Unaligned/Project_agreaney/expt_149-151_lib2_ref-4_S8_R1_001.fastq.gz,expt_149-151_lib2_ref-4_S8_R1_001.fastq.gz


For the actual submission, we need a "wide" data frame that for each unique `sample_name` / `library_ID` gives all of the files each in different columns.
These should be files without the full path.

First, look at how many files there are for each sample / library:

In [6]:
(submissions_tidy
 .groupby(['biosample_accession', 'library_ID'])
 .aggregate(n_files=pd.NamedAgg('filename_fullpath', 'count'))
 .sort_values('n_files', ascending=False)
 .reset_index()
 )

,biosample_accession,library_ID,n_files
0,SAMN26873633,lib1_expt_149-151-none-0-reference,4
1,SAMN26873633,lib2_expt_149-151-none-0-reference,4
2,SAMN26873633,lib1_expt_149-CAB-A17-139-escape,1
3,SAMN26873633,lib1_expt_150-CAB-A49-222-escape,1
4,SAMN26873633,lib1_expt_151-CAB-C19-184-escape,1
5,SAMN26873633,lib2_expt_149-CAB-A17-139-escape,1
6,SAMN26873633,lib2_expt_150-CAB-A49-222-escape,1
7,SAMN26873633,lib2_expt_151-CAB-C19-184-escape,1


Now make the wide submission data frame.
Note we keep only the filename column with the path lacking the full directory information:

In [7]:
submissions_wide = (
    submissions_tidy
    .assign(
        filename_count=lambda x: x.groupby(['biosample_accession', 'library_ID'])['filename'].cumcount() + 1,
        filename_col=lambda x: 'filename' + x['filename_count'].map(lambda c: str(c) if c > 1 else '')
        )
    .pivot(
        index='library_ID',
        columns='filename_col',
        values='filename',
        )
    )

submissions_wide = (
    submissions_tidy
    .drop(columns=['filename_fullpath', 'filename'])
    .drop_duplicates()
    .merge(submissions_wide[natsort.natsorted(submissions_wide.columns)],
           on='library_ID',
           validate='one_to_one',
           )
    )

print('Here are the first few submission entries in wide format:')
submissions_wide.head()

Here are the first few submission entries in wide format:


,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename,filename2,filename3,filename4
0,SAMN26873633,lib1_expt_149-151-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_149-151-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,expt_149-151_lib1_ref-3_S3_R1_001.fastq.gz,expt_149-151_lib1_ref-1_S1_R1_001.fastq.gz,expt_149-151_lib1_ref-2_S2_R1_001.fastq.gz,expt_149-151_lib1_ref-4_S4_R1_001.fastq.gz
1,SAMN26873633,lib2_expt_149-151-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_149-151-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,expt_149-151_lib2_ref-4_S8_R1_001.fastq.gz,expt_149-151_lib2_ref-3_S7_R1_001.fastq.gz,expt_149-151_lib2_ref-2_S6_R1_001.fastq.gz,expt_149-151_lib2_ref-1_S5_R1_001.fastq.gz
2,SAMN26873633,lib1_expt_149-CAB-A17-139-escape,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_149-CAB-A17-139-escape,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,expt_149_lib1_abneg_S25_R1_001.fastq.gz,NaN,NaN,NaN
3,SAMN26873633,lib2_expt_149-CAB-A17-139-escape,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_149-CAB-A17-139-escape,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,expt_149_lib2_abneg_S26_R1_001.fastq.gz,NaN,NaN,NaN
4,SAMN26873633,lib1_expt_150-CAB-A49-222-escape,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_150-CAB-A49-222-escape,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,expt_150_lib1_abneg_S27_R1_001.fastq.gz,NaN,NaN,NaN


Now write the wide submissions data frame to a `*.tsv` file that can be used for uploading the SRA submission website:

In [8]:
submissions_spreadsheet = 'SRA_submission_spreadsheet.tsv'

submissions_wide.to_csv(submissions_spreadsheet, sep='\t', index=False)

We also want to write a file with all of the FASTQ files that are in the submission spreadsheet:

In [9]:
fastq_file_list = 'FASTQs_to_upload.csv'

print(f"Writing list of all the FASTQ files to upload to {fastq_file_list}")

(submissions_tidy
 [['filename_fullpath', 'filename']]
 .to_csv(fastq_file_list, index=False)
 )

Writing list of all the FASTQ files to upload to FASTQs_to_upload.csv
